The model includes both an offensive and defensive universe:
- Offensive: US equities (represented by SPY), international equities (EFA), emerging market equities (EEM) and US aggregate bonds (AGG). More on the seemingly odd inclusion of AGG as an offensive asset in a moment.
- Defensive: US corporate bonds (LQD), US intermediate-term Treasuries (IEF) and US short-term Treasuries (SHY).
- For all assets, at the close on the last trading day of the month, calculate a “momentum score” based on month-end data as follows:
$$12(\frac{p_0}{p_1} – 1) + 4(\frac{p_0}{p_3} – 1) + 2(\frac{p_0}{p_6} – 1) + (\frac{p_0}{p_{12}} – 1)$$
 - Where p0 = the asset price at today’s close, p1 = the asset price at the close of the previous month, etc.
 - Note how this approach overweights more recent months. Doing the math, the most recent 1-month change (p0/p1 – 1) determines 40% of the momentum score, while the most distant month (p11/p12 – 1) determines just ~2%.

- If all four of the offensive assets exhibit positive momentum scores, select the offensive asset with the highest score and allocate 100% of the portfolio to that asset at the close. Note the use of both absolute and relative momentum here, an idea popularized by Gary Antonacci as “Dual Momentum”. Why is that important? Historically, absolute momentum has done well minimizing losses, while relative momentum has helped in generating outsized returns.
- If any of the four offensive assets exhibit negative momentum scores, select the defensive asset (LQD, IEF or SHY) with the highest score (regardless of whether the score is > 0) and allocate 100% of the portfolio to that asset at the close. As we do throughout this site, trades in SHY are assumed to be placed in cash, as it’s more relevant to today’s market given SHY’s low yields coupled with the impact of transaction costs and how frequently this strategy trades.
- Hold the position until the final trading day of the following month.

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri May 25 22:22:16 2018

@author: xexx
"""

import pandas as pd
import numpy as np
import pickle

#pull in data from AlphaVantage
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='NXY0VT9AHBRYGKKC',output_format='pandas')




In [2]:
def historical_data(ticker, outsize = "full"):
    #outsize can be compact and full. 
    #compact returns only the latest 100 data points; full 
    #returns the full-length time series of up to 20 years of historical data. The "compact" option is 
    #recommended if you would like to reduce the data size of each API call
    alphavantage_link = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={0}&apikey=NXY0VT9AHBRYGKKC&datatype=csv&outputsize={1}'.format(ticker, outsize)
    df = pd.read_csv(alphavantage_link)
    return df


In [3]:
VAA = ['AGG', 'EFA', 'EEM']
Maindf = historical_data("SPY")
Maindf.index = pd.to_datetime(Maindf.timestamp) #set the index as the timestamp
#Maindf.index = pd.to_datetime(Maindf.index) #make the index datetime
Maindf = Maindf[['timestamp','close']]
Maindf.columns = ["timestamp","SPY"]


In [4]:
for ticker in VAA:
    temp_df = historical_data(ticker)
    temp_df.index = pd.to_datetime(temp_df.timestamp) #set the index as the timestamp
    temp_df = temp_df[['close']]
    temp_df.columns = [ticker]
    Maindf = Maindf.join(temp_df, how = "left")

Maindf.dropna(inplace = True)
Maindf = Maindf.sort_index()

In [5]:
Maindf = Maindf.sort_index()

In [6]:
Maindf.head(5)

,timestamp,SPY,AGG,EFA,EEM
timestamp,,,,,
2003-09-26,2003-09-26,99.95,102.45,117.00,135.40
2003-09-29,2003-09-29,100.93,102.17,118.50,136.36
2003-09-30,2003-09-30,99.95,102.70,117.00,135.93
2003-10-01,2003-10-01,102.08,102.65,120.84,139.50
2003-10-02,2003-10-02,102.45,102.49,120.65,141.50


In [7]:
Maindf['Year'] = Maindf.index.year
Maindf['Month'] = Maindf.index.month


In [8]:
Maindf['Month Change'] = Maindf['Month'] - Maindf['Month'].shift(1) #shift 1 is to shift down 

In [44]:
Maindf_EOM = Maindf[Maindf['Month Change'] != 0]
Maindf_EOM = Maindf_EOM[['SPY','AGG','EFA','EEM']]

In [11]:
Maindf_EOM.to_csv('VAA.csv')

In [52]:
Maindf_EOM_VAA = Maindf_EOM.pct_change(12) + 2 * Maindf_EOM.pct_change(6) + 4 * Maindf_EOM.pct_change(3) + 12 * Maindf_EOM.pct_change(1)

In [53]:
Maindf_EOM_VAA

,SPY,AGG,EFA,EEM
timestamp,,,,
2003-09-26,NaN,NaN,NaN,NaN
2003-10-01,NaN,NaN,NaN,NaN
2003-11-03,NaN,NaN,NaN,NaN
2003-12-01,NaN,NaN,NaN,NaN
2004-01-02,NaN,NaN,NaN,NaN
2004-02-02,NaN,NaN,NaN,NaN
2004-03-01,NaN,NaN,NaN,NaN
2004-04-01,NaN,NaN,NaN,NaN
2004-05-03,NaN,NaN,NaN,NaN


In [13]:
with open('filename.pickle', 'wb') as handle:
    pickle.dump(Maindf, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
with open('filename.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [15]:
import volatility as vm

In [16]:
b.tail(10)

,timestamp,SPY,AGG,EFA,EEM,Year,Month,Month Change
timestamp,,,,,,,,
2018-05-17,2018-05-17,272.01,105.03,71.59,46.46,2018,5,0.0
2018-05-18,2018-05-18,271.33,105.30,71.33,46.07,2018,5,0.0
2018-05-21,2018-05-21,273.37,105.31,71.78,46.35,2018,5,0.0
2018-05-22,2018-05-22,272.61,105.31,71.69,46.51,2018,5,0.0
2018-05-23,2018-05-23,273.36,105.60,70.90,46.49,2018,5,0.0
2018-05-24,2018-05-24,272.80,105.89,70.68,46.24,2018,5,0.0
2018-05-25,2018-05-25,272.15,106.12,70.22,46.43,2018,5,0.0
2018-05-29,2018-05-29,269.02,106.82,68.79,45.35,2018,5,0.0
2018-05-30,2018-05-30,272.61,106.55,69.84,45.71,2018,5,0.0
